In [13]:
# Importing necessary libraries
import mlflow
from mlflow.tracking import MlflowClient
from fastapi import FastAPI, UploadFile, File
from PIL import Image
import io
from keras.models import load_model, save_model
from tensorflow import keras
import os
import numpy as np
import matplotlib.pyplot as plt
import random

In [14]:
client = MlflowClient(tracking_uri='http://localhost:8080') 
experiments = client.search_experiments()

In [15]:
for experiment in experiments:
    print("Experiment ID:", experiment.experiment_id)
    print("Name:", experiment.name)
    print("Artifact Location:", experiment.artifact_location)
    print("Lifecycle Stage:", experiment.lifecycle_stage)
    print("-------------------------------")

Experiment ID: 546648210945176913
Name: MNIST models auto
Artifact Location: mlflow-artifacts:/546648210945176913
Lifecycle Stage: active
-------------------------------
Experiment ID: 616345290327544943
Name: MNIST models manual
Artifact Location: mlflow-artifacts:/616345290327544943
Lifecycle Stage: active
-------------------------------
Experiment ID: 0
Name: Default
Artifact Location: mlflow-artifacts:/0
Lifecycle Stage: active
-------------------------------


In [16]:
experiment_id = '546648210945176913'

runs = mlflow.search_runs(experiment_id)


In [17]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time


In [18]:
for run in runs.iterrows():
    print("Run ID:", run[1]['run_id'])
    print("Status:", run[1]['status'])
    print("Start Time:", run[1]['start_time'])
    print("End Time:", run[1]['end_time'])
    print("Validation Loss:",run[1]['metrics.val_loss'])
    print("-------------------------------------")

In [6]:
# Picking the model with the least validation loss

best_loss = float('inf')
best_run_id = None

for run in runs.iterrows():
    loss = run[1]['metrics.val_loss']

    if loss < best_loss:
        best_loss = loss
        best_run_id = run[1]['run_id']

print("Best Run ID:", best_run_id)
print("Best Validation Loss:", best_loss)

Best Run ID: None
Best Validation Loss: inf


In [7]:
model = mlflow.keras.load_model("runs:/" + best_run_id + "/model")

TypeError: can only concatenate str (not "NoneType") to str

In [45]:
# Saving the model on the disk

def save_model_to_disk(model,file_path,model_name):
    os.makedirs(file_path,exist_ok=True)
    save_path = os.path.join(file_path,model_name)

    save_model(model,save_path)

save_model_to_disk(model,'Models','MNIST_model.keras')
